In [ ]:
!pip install '/content/pyGPB-0.0.1.tar.gz'

Processing ./pyGPB-0.0.1.tar.gz
  Created wheel for pyGPB: filename=pyGPB-0.0.1-py3-none-any.whl size=11717 sha256=a6093df8089969afdf70ef9c53e8ed79cdd3511ac1b36a99acc3159e241d1142
  Stored in directory: /root/.cache/pip/wheels/18/9e/73/f0be66f2d221d99a8ecad80a5f64821626f2a6c13e8f039eab
Successfully built pyGPB


In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-397c7cf2-3009-c3ec-91dc-7b09d8405c29)


In [ ]:
cpuinfomeminfo

MemTotal:       13302928 kB
MemFree:        10404004 kB
MemAvailable:   12492288 kB
Buffers:          150724 kB
Cached:          2065384 kB
SwapCached:            0 kB
Active:          1070144 kB
Inactive:        1586520 kB
Active(anon):     398960 kB
Inactive(anon):      456 kB
Active(file):     671184 kB
Inactive(file):  1586064 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               416 kB
Writeback:             0 kB
AnonPages:        440468 kB
Mapped:           269544 kB
Shmem:              1144 kB
KReclaimable:     145020 kB
Slab:             190780 kB
SReclaimable:     145020 kB
SUnreclaim:        45760 kB
KernelStack:        5648 kB
PageTables:         6340 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6651464 kB
Committed_AS:    3580376 kB
VmallocTotal:   34359738367 kB
VmallocUsed:        8152 kB
VmallocChunk:          0 kB
Percpu:          

In [ ]:
import numpy as np
import pickle
import timeit
import numba
import math
#from pyGPB._cpu_methods import _GPB_CF

In [ ]:
# Test space definition
NUMBER_OF_TEST_CASES_PER_SCENARIO = 16
M_space = np.logspace(1,4,num=25, dtype=np.int32)
N_space = np.logspace(3,7,num=25, dtype=np.int32)
weight_distribution_space = ['uniform', 'poisson', 'geometric']
prob_distribution_space = ['uniform', 'near_0', 'near_1']
N_space, M_space, weight_distribution_space, prob_distribution_space

(array([    1000,     1467,     2154,     3162,     4641,     6812,
           10000,    14677,    21544,    31622,    46415,    68129,
          100000,   146779,   215443,   316227,   464158,   681292,
         1000000,  1467799,  2154434,  3162277,  4641588,  6812920,
        10000000], dtype=int32),
 array([   10,    13,    17,    23,    31,    42,    56,    74,   100,
          133,   177,   237,   316,   421,   562,   749,  1000,  1333,
         1778,  2371,  3162,  4216,  5623,  7498, 10000], dtype=int32),
 ['uniform', 'poisson', 'geometric'],
 ['uniform', 'near_0', 'near_1'])

In [ ]:
# Load test data
test_case_weights = pickle.load( open( '/content/drive/MyDrive/MSc Project/test_cases_weights.p', "rb" ) )
test_case_probs = pickle.load( open( '/content/drive/MyDrive/MSc Project/test_cases_probabilities.p', "rb" ) )

In [ ]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2200.140
BogoMIPS:            4400.28
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

In [ ]:
!free -h --si | awk  '/Mem:/{print $2}'

13G


In [ ]:
!sudo lshw -short -C memory

sudo: lshw: command not found


In [ ]:
weight_distribution = 'uniform'
prob_distribution = 'uniform'
for N in [1000,10000,100000,1000000,10000000]:
    for M in M_space:
        if N > M:
            times = dict()
            for func in ['_GPB_CF']:
                initial_time_estimate = timeit.timeit(f"{func}(test_case_probs[('{prob_distribution}', {M})][0], test_case_weights[('{weight_distribution}', {N}, {M})][0])", globals=globals(), number=1)
                if initial_time_estimate > 1.0:
                    best_time = initial_time_estimate
                else:
                    approx_runs_in_1_second = 1 / initial_time_estimate
                    if approx_runs_in_1_second < NUMBER_OF_TEST_CASES_PER_SCENARIO**2:
                        cases_to_use = int(math.sqrt(approx_runs_in_1_second))
                    else:
                        cases_to_use = NUMBER_OF_TEST_CASES_PER_SCENARIO
                    repeats = max(int(approx_runs_in_1_second // cases_to_use),1)
                    statement = f"""for probs, weights in zip(probs_cases, weights_cases): {func}(probs, weights)
                    """
                    setup = f"""probs_cases = test_case_probs[('{prob_distribution}', {M})][:{cases_to_use}]\nweights_cases = test_case_weights[('{weight_distribution}', {N}, {M})][:{cases_to_use}]
                    """
                    timer = timeit.Timer(statement, setup=setup, globals=globals())
                    results = timer.repeat(repeat=repeats, number=1)
                    best_time = min(results)/cases_to_use
                times[func] = best_time
            print(N, M, times['_GPB_CF'])

1000 10 0.0005061479999994845
1000 13 0.0005138253750001098
1000 17 0.0006517311250000546
1000 23 0.0008725650625009962
1000 31 0.0011581805625002772
1000 42 0.0015527618750006411
1000 56 0.002043460375000805
1000 74 0.0026517453750010134
1000 100 0.003515719937499995
1000 133 0.004575833833333339
1000 177 0.005873246307693045
1000 237 0.007604334999999196
1000 316 0.009777691777780243
1000 421 0.012783365249998013
1000 562 0.016613982714285873
1000 749 0.020760638666663073
10000 10 0.004232360857143037
10000 13 0.005347650583334011
10000 17 0.006754213700000377
10000 23 0.008883303888890422
10000 31 0.011929329111113576
10000 42 0.015690111714288184
10000 56 0.02078694866666808
10000 74 0.027491530400004648
10000 100 0.03628167940000253
10000 133 0.04811992824999578
10000 177 0.0627786600000017
10000 237 0.08374464533333519
10000 316 0.1106331105000038
10000 421 0.14844851750000032
10000 562 0.19864547900000673
10000 749 0.261719869999979
10000 1000 0.34281212000001915
10000 1333 0.45